In [1]:
import pyomo.environ as pyo
import pandas as pd
from pathlib import Path
import preprocess_data as dat

known_x_sols = [[8, 14, 17, 19, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87],
              [4, 8, 14, 18, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87],
              [8, 14, 17, 18, 29, 33, 35, 38, 51, 55, 59, 68, 75, 78, 87]]

## Data

In [2]:
data_path = Path("../data")
year = 2021

adjacent_list = dat.create_adjacent_list(data_path)
adjacent_matrix = dat.gen_adjacent_matrix(adjacent_list)
df_init, counties = dat.data_ingest(data_path, year)
df = dat.create_df(df_init, counties, adjacent_list)
model = pyo.AbstractModel()

## Defs

In [3]:
def param_adjacent(m, i, j):
    return int(j in adjacent_matrix[i])

def con_a(m, i):
    return sum((m.a[i, j] * m.x[j]) for j in m.J) >= 1

def obj_sum(m):
    return pyo.summation(m.x)

## Constraints

In [4]:
# value of n (number of counties)
model.n = 88

# range of i and j (iterating over counties)
model.I = pyo.RangeSet(1, model.n)
model.J = pyo.RangeSet(1, model.n)

# model.a = pyo.Set(model.I, model.J, within=pyo.Binary, initialize=param_adjacent)  # 1 if county i and j are adjacent
model.a = pyo.Param(model.I, model.J, domain=pyo.Binary, initialize=param_adjacent)  # 1 if county i and j are adjacent

model.x = pyo.Var(model.J, domain=pyo.Binary)  # 1 if principal place of business is opened in county j

model.obj = pyo.Objective(rule=obj_sum, sense=pyo.minimize)

model.a_constraint = pyo.Constraint(model.I, rule=con_a)

## Solve

In [5]:
instance = model.create_instance()
# opt = pyo.SolverFactory("ipopt", executable="/home/adb/anaconda3/bin/ipopt")
opt = pyo.SolverFactory("glpk", executable="/home/adb/anaconda3/bin/glpsol")
results = opt.solve(instance).write()
results
# instance.pprint()
# instance.solutions.store_to(results)
# model21.pprint()
# model21.x.get_values()
# solution.print_data()
# solution.solutions.store_to(results)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 16.0
  Upper bound: 16.0
  Number of objectives: 1
  Number of constraints: 89
  Number of variables: 89
  Number of nonzeros: 543
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 39
      Number of created subproblems: 39
  Error rc: 0
  Time: 0.010195255279541016
# ----------------------------------------------------------
#   Solution Information
# --------------------------------

In [13]:
sol_dict = instance.x.get_values()
sol1 = [k for k, v in sol_dict.items() if v == 1]
sol1

[5, 6, 7, 17, 19, 20, 28, 31, 36, 45, 47, 48, 49, 73, 76, 87]